<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling

In [69]:
import warnings
warnings.filterwarnings("ignore")

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.feature_extraction.text import TfidfVectorizer

import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.tokenizer import Tokenizer

import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
nlp = spacy.load("en_core_web_lg", disable=['parser', 'tagger', 'ner'])

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
tokenizer = Tokenizer(nlp.vocab)

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df1 = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
print(df1.shape)
df1.head()

(28332, 24)


C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [6]:
df1['primaryCategories'].value_counts()

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Electronics                    13995
Health & Beauty                12071
Toys & Games,Electronics        1676
Office Supplies,Electronics      386
Electronics,Media                185
Office Supplies                    9
Animals & Pet Supplies             6
Home & Garden                      2
Electronics,Furniture              2
Name: primaryCategories, dtype: int64

In [7]:
# df2 = df1[df1['primaryCategories'] == 'Office Supplies,Electronics'].copy()
df2 = df1.copy()

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
df2.shape

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(28332, 24)

In [9]:
df=df2[['reviews.text', 'reviews.rating']]

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
pd.set_option('display.max_colwidth',300)
print(df.shape)
df.head()

(28332, 2)


C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,reviews.text,reviews.rating
0,I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.,3
1,Bulk is always the less expensive way to go for products like these,4
2,Well they are not Duracell but for the price i am happy.,5
3,Seem to work as well as name brand batteries at a much better price,5
4,These batteries are very long lasting the price is great.,5


In [11]:
import re
# For reference on regex: https://docs.python.org/3/library/re.html
# 1. Remove new line characters
df['clean_text'] = df['reviews.text'].apply(lambda x: re.sub('\s+', ' ', x))

# 2. Remove Emails
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('From: \S+@\S+', '', x))

# 3. Remove non-alphanumeric characters
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

# 4. Remove extra whitespace and lowercase text
df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join(x.lower().split()))

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:4: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \S
<>:4: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \S
<ipython-input-11-de7420d041cc>:4: DeprecationWarning: invalid escape sequence \s
  df['clean_text'] = df['reviews.text'].apply(lambda x: re.sub('\s+', ' ', x))
<ipython-input-11-de7420d041cc>:7: DeprecationWarning: invalid escape sequence \S
  df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('From: \S+@\S+', '', x))


In [12]:
print(df.shape)
df.sample(5)

(28332, 3)


C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,reviews.text,reviews.rating,clean_text
13126,"I bought this game for my son, and he loves it..it is very entertaining.",4,i bought this game for my son and he loves it it is very entertaining
25333,"Tablet is great. I am trying to get my daughter use to the kindle. We started her at 10 months on an Ipad and thought it would be easier to get one her own tablet now that she is almost 2. Not so, she is struggling to navigate in the kindle. I am not able to transfer her games and movies.",3,tablet is great i am trying to get my daughter use to the kindle we started her at months on an ipad and thought it would be easier to get one her own tablet now that she is almost not so she is struggling to navigate in the kindle i am not able to transfer her games and movies
1128,"Good price, good quality and shipping time much less expensive than at the local drugstore. will buy again..",5,good price good quality and shipping time much less expensive than at the local drugstore will buy again
17002,"This is a good product for video, surfing and reading. However this version has dropped all android support as far as I can tell. I have a couple of apps that run on my old Fire just fine but are extremely problematic on this one. There is one app I use everyday but it doesn't work on this new F...",3,this is a good product for video surfing and reading however this version has dropped all android support as far as i can tell i have a couple of apps that run on my old fire just fine but are extremely problematic on this one there is one app i use everyday but it doesn t work on this new fire ...
14384,Enjoy reading books on it as well as viewing videos.,4,enjoy reading books on it as well as viewing videos


In [13]:
# # Leverage tqdm for progress_apply
from tqdm import tqdm
tqdm.pandas()

# Create 'lemmas' column
def get_lemmas(x):
    lemmas = []
    for token in nlp(x):
        if (token.is_stop != True) and (token.is_punct != True) and (len(token) > 2):
            lemmas.append(token.lemma_)
    return lemmas
df['lemmas'] = df['clean_text'].progress_apply(get_lemmas)
df.head(3)

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████████████████████████████████████████████████████████████████████| 28332/28332 [00:07<00:00, 3637.43it/s]


,reviews.text,reviews.rating,clean_text,lemmas
0,I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.,3,i order of them and one of the item is bad quality is missing backup spring so i have to put a pcs of aluminum to make the battery work,"[order, item, bad, quality, miss, backup, spring, pcs, aluminum, battery, work]"
1,Bulk is always the less expensive way to go for products like these,4,bulk is always the less expensive way to go for products like these,"[bulk, expensive, way, product, like]"
2,Well they are not Duracell but for the price i am happy.,5,well they are not duracell but for the price i am happy,"[duracell, price, happy]"


In [31]:
from collections import Counter

# Introduce count function
def count(docs):
    '''This function takes a list of tokenized documents as input and returns
    a dataframe with frequency counts and rank for each token.
    
    # Arguments
        docs: list, tokenized list of documents
        
    # Returns
        wc: dataframe, 
    '''
    
    word_counts = Counter()
    appears_in = Counter()

    total_docs = len(docs)

    for doc in docs:
        word_counts.update(doc)
        appears_in.update(set(doc))

    temp = zip(word_counts.keys(), word_counts.values())

    wc = pd.DataFrame(temp, columns = ['word', 'count'])

    wc['rank'] = wc['count'].rank(method='first', ascending=False)
    total = wc['count'].sum()

    wc['pct_total'] = wc['count'].apply(lambda x: x / total)

    wc = wc.sort_values(by='rank')
    wc['cul_pct_total'] = wc['pct_total'].cumsum()

    t2 = zip(appears_in.keys(), appears_in.values())
    ac = pd.DataFrame(t2, columns=['word', 'appears_in'])
    wc = ac.merge(wc, on='word')

    wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)

    return wc.sort_values(by='rank')

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
# Use the Function
wc = count(df['lemmas'])
print(wc.shape)
wc.head(30)

(7036, 7)


C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,word,appears_in,count,rank,pct_total,cul_pct_total,appears_in_pct
9,battery,6691,10280,1.0,0.032157,0.032157,0.236164
23,great,8224,9512,2.0,0.029755,0.061912,0.290272
3166,tablet,5615,7411,3.0,0.023182,0.085094,0.198186
86,love,5552,6232,4.0,0.019494,0.104589,0.195962
39,buy,5141,5994,5.0,0.018750,0.123339,0.181456
29,good,5224,5907,6.0,0.018478,0.141816,0.184385
16,price,5160,5495,7.0,0.017189,0.159005,0.182126
85,use,4261,4866,8.0,0.015221,0.174227,0.150395
7,work,4254,4635,9.0,0.014499,0.188726,0.150148
24,amazon,3149,3945,10.0,0.012340,0.201066,0.111146


In [50]:
# SpaCy Tokenizer requires a string, lemmas is a list
# prepare to add custom stop words
df['lem_string'] = df['lemmas'].apply(lambda x: ' '.join(map(str, x)))
df['lem_string'].head()

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    order item bad quality miss backup spring pcs aluminum battery work
1                                        bulk expensive way product like
2                                                   duracell price happy
3                                          work brand battery well price
4                                          battery long last price great
Name: lem_string, dtype: object

In [51]:
STOP_WORDS = ['get', 'buy', 'purchase', 'use', 'product', 'item']
tokens = []
for doc in tokenizer.pipe(df['lem_string'], batch_size=500):
    
    doc_tokens = []
    for token in doc:
        if (token.is_alpha == True)& (token.text.lower() not in STOP_WORDS):
            doc_tokens.append(token.text.lower())
   
    tokens.append(doc_tokens)
    
df['lemmas_refined'] = tokens
df['lemmas_refined'].head()

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    [order, bad, quality, miss, backup, spring, pcs, aluminum, battery, work]
1                                                 [bulk, expensive, way, like]
2                                                     [duracell, price, happy]
3                                          [work, brand, battery, well, price]
4                                          [battery, long, last, price, great]
Name: lemmas_refined, dtype: object

In [52]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemmas_refined'])

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in df['lemmas_refined']]

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
# How many words in our vocab?
len(id2word.keys())

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


7023

In [67]:
# Human readable format of corpus (term-frequency)
[(id2word[word_id], word_count) for word_id, word_count in corpus[28]]

C:\Users\whats\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('order', 1),
 ('good', 1),
 ('year', 1),
 ('time', 1),
 ('chance', 1),
 ('hopefully', 1),
 ('mean', 1),
 ('reasonable', 1),
 ('remain', 1)]

In [75]:
%%time
lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics=4, 
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=3)


Wall time: 1min 48s


In [76]:
#below is essentially the same as pickling
lda_multicore.save('lda_multicore.model')

In [77]:
lda_multicore =  models.LdaModel.load('lda_multicore.model')

In [78]:
pprint(lda_multicore.print_topics())

[(0,
  '0.241*"great" + 0.119*"good" + 0.100*"price" + 0.060*"work" + 0.024*"nice" '
  '+ 0.020*"quality" + 0.019*"fast" + 0.018*"recommend" + 0.015*"value" + '
  '0.014*"well"'),
 (1,
  '0.035*"battery" + 0.029*"amazon" + 0.025*"long" + 0.023*"store" + '
  '0.017*"work" + 0.016*"well" + 0.015*"don" + 0.013*"excellent" + '
  '0.013*"life" + 0.012*"time"'),
 (2,
  '0.066*"love" + 0.062*"tablet" + 0.033*"kid" + 0.030*"old" + 0.029*"fire" + '
  '0.025*"year" + 0.021*"easy" + 0.020*"kindle" + 0.018*"game" + 0.014*"play"'),
 (3,
  '0.040*"tablet" + 0.035*"app" + 0.034*"read" + 0.027*"book" + 0.024*"screen" '
  '+ 0.020*"kindle" + 0.019*"easy" + 0.016*"amazon" + 0.015*"perfect" + '
  '0.014*"like"')]


In [79]:
# Compute Perplexity
print('\nPerplexity: ', lda_multicore.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_multicore, 
                                     texts=df['lemmas_refined'], 
                                     dictionary=id2word, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.497188231143978

Coherence Score:  0.5557231357934463


In [80]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_multicore, corpus, id2word)
pyLDAvis.display(vis)

## Observations

* With 8 topics there was minimal overlap
* Dropping 6 custom stop words reducted the number to 4 distinct topics
    1 - Use of tablet/kindle by children
    2 - Batteries
    3 - Various fuctions/uses of tablet/kindle
    4 - Positive reviews related to value & price
* Analyze a dataset of interest to you with topic modeling